# Анализ дампа памяти или сетевого трафика
## Практическая работа по расследованию инцидента

### Цель работы
1. Загрузить дамп памяти или сетевого трафика
2. Извлечь ключевые артефакты
3. Выполнить базовую аналитику
4. Построить визуализацию
5. Сформировать краткие выводы

---
В этой работе вы действуете как аналитик информационной безопасности.
Ваша задача — не просто получить данные, а выявить потенциальные признаки компрометации.


# ЭТАП 1. Загрузка данных

Выберите один из вариантов:

## Вариант A — Анализ дампа памяти (Volatility)

### 1. Определение информации о системе
```bash
vol.py -f memory_dump.raw windows.info
```

Это позволяет определить профиль ОС. Неверный профиль приведёт к некорректным результатам анализа.

### 2. Получение списка процессов
```bash
vol.py -f memory_dump.raw windows.pslist
```

Обратите внимание на:
- PID и PPID
- Время запуска
- Необычные имена процессов
- Процессы без родителя

### 3. Получение сетевых соединений
```bash
vol.py -f memory_dump.raw windows.netscan
```

Необходимо связать процесс → PID → IP → порт.
Особое внимание уделяется внешним IP-адресам и состоянию ESTABLISHED.


## Вариант B — Анализ сетевого дампа (PCAP)

### Установка библиотек
```python
!pip install pyshark pandas matplotlib
```

DNS-запросы особенно важны, так как:
- используются практически всеми вредоносными программами
- позволяют выявить beaconing
- могут указывать на DGA-домены


In [ ]:
import pyshark
import pandas as pd

pcap_file = 'sample.pcap'
cap = pyshark.FileCapture(pcap_file)


# ЭТАП 2. Извлечение ключевых артефактов

Извлекаем DNS-запросы, IP-адреса и временные метки.
Это позволит построить таймлайн активности.


In [ ]:
dns_requests = []

for packet in cap:
    try:
        if 'DNS' in packet:
            dns_requests.append({
                'time': packet.sniff_time,
                'domain': packet.dns.qry_name,
                'src_ip': packet.ip.src,
                'dst_ip': packet.ip.dst
            })
    except AttributeError:
        continue

df_dns = pd.DataFrame(dns_requests)
df_dns.head()


## Частотный анализ доменов
Позволяет выявить наиболее часто запрашиваемые домены.
Необычно высокая частота обращений может свидетельствовать о C2 или beaconing.


In [ ]:
domain_counts = df_dns['domain'].value_counts()
domain_counts.head(10)


# ЭТАП 3. Визуализация

Анализ временного распределения DNS-запросов.
Регулярная периодичность может указывать на автоматизированную активность.


In [ ]:
import matplotlib.pyplot as plt

df_dns['time'] = pd.to_datetime(df_dns['time'])
df_dns['minute'] = df_dns['time'].dt.floor('T')

dns_by_time = df_dns.groupby('minute').size()

plt.figure()
dns_by_time.plot()
plt.xlabel('Время')
plt.ylabel('Количество DNS-запросов')
plt.title('DNS-запросы по времени')
plt.show()


# Формирование выводов

Ответьте на вопросы:
1. Есть ли подозрительные процессы или соединения?
2. Есть ли неизвестные или часто повторяющиеся домены?
3. Наблюдается ли периодичность запросов?
4. Можно ли сформулировать гипотезу атаки?

Структура краткого отчёта:
- Источник данных
- Метод анализа
- Найденные артефакты
- Предварительный вывод
- Рекомендации


# Типичные ошибки

1. Неверный профиль ОС при анализе памяти.
2. Игнорирование временной корреляции событий.
3. Оценка доменов без проверки частоты обращений.
4. Отсутствие связи между процессом и сетевым соединением.
5. Формирование выводов без доказательной базы.
6. Отсутствие сохранения результатов анализа.

Важно помнить: аналитик должен опираться на артефакты, а не на предположения.
